# Plotting code for making figures

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import time
import open3d as o3d
import plotly.graph_objects as go

import planeslam.general as general
from planeslam.mesh import LidarMesh
from planeslam.scan import Scan
from planeslam.clustering import cluster_mesh_graph_search, plot_clusters
from planeslam.extraction import scan_from_clusters
from planeslam.scan import pc_to_scan
import planeslam.io as io

%load_ext autoreload
%autoreload 2
%load_ext line_profiler

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Some example data to plot

In [2]:
binpath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'lidar', 'Drone0')
PCs = io.read_lidar_bin(binpath)

In [ ]:
P = PCs[7]

# Convert points to ENU
P = general.NED_to_ENU(P)

In [ ]:
scan = pc_to_scan(P)

Plot without axes/grid

In [ ]:
fig = go.Figure(data=scan.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)))
fig.show()

2D trajectory plot

In [ ]:
gt_traj_trace_2d = go.Scatter(x=rover_positions[:,0], y=rover_positions[:,1], mode='lines+markers',
    marker=dict(size=5), hovertext=np.arange(len(rover_positions)))
gt_traj_trace_2d.name = "Ground-truth"
est_traj_trace_2d = go.Scatter(x=positions[:,0], y=positions[:,1], mode='lines+markers',
    marker=dict(size=5), hovertext=np.arange(len(positions)))
est_traj_trace_2d.name = "Estimated"
fig = go.Figure(data=[gt_traj_trace_2d, est_traj_trace_2d])
fig.update_layout(width=1500, height=600, scene=dict(aspectmode='data'),
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()

Fix camera angle

In [ ]:
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=-1.75, z=1.75)
)

fig = go.Figure(data=merged.plot_trace(colors=['blue'])+[gt_traj_trace])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data', xaxis=dict(showticklabels=False, title=dict(text='')), 
    yaxis=dict(showticklabels=False, title=dict(text='')), zaxis=dict(showticklabels=False, title=dict(text=''))),
    scene_camera=camera)
fig.show()

TODO: export vector images
https://plotly.com/python/static-image-export/

In [3]:
from planeslam.geometry.plane import BoundedPlane

In [40]:
B = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

In [30]:
from scipy.spatial.transform import Rotation as R

r = R.from_euler('xy', [30, 45], degrees=True)
B = r.as_matrix()

In [33]:
B

array([[ 7.07106781e-01,  3.53553391e-01,  6.12372436e-01],
       [-2.77555756e-17,  8.66025404e-01, -5.00000000e-01],
       [-7.07106781e-01,  3.53553391e-01,  6.12372436e-01]])

In [41]:
V = np.array([-B[:,0] - B[:,1], 
               B[:,0] - B[:,1],
               B[:,0] + B[:,1],
              -B[:,0] + B[:,1]])
p = BoundedPlane(V)

In [42]:
cone = go.Cone(x=[B[0,2]], y=[B[1,2]], z=[B[2,2]], u=[B[0,2]], v=[B[1,2]], w=[0.4*B[2,2]], colorscale=[[0, 'rgb(0,0,0)'], [1, 'rgb(0,0,0)']])
lines = go.Scatter3d(x=[0,B[0,0],None,0,B[0,1],None,0,B[0,2]], y=[0,B[1,0],None,0,B[1,1],None,0,B[1,2]], z=[0,B[2,0],None,0,B[2,1],None,0,B[2,2]], 
    mode="lines", line=dict(color="black",width=5), showlegend=False)

fig = go.Figure(data=p.plot_trace() + [lines])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data', 
    xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)))
fig.show()